In [1]:
import cv2
import numpy as np
import scipy.fftpack

def apply_dct(image):
    """Apply DCT to an 8x8 block."""
    return scipy.fftpack.dct(scipy.fftpack.dct(image, axis=0, norm='ortho'), axis=1, norm='ortho')

def apply_idct(dct_block):
    """Apply Inverse DCT to an 8x8 block."""
    return scipy.fftpack.idct(scipy.fftpack.idct(dct_block, axis=0, norm='ortho'), axis=1, norm='ortho')

def embed_message(image_path, secret_message):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    h, w = image.shape

    binary_msg = ''.join(format(ord(char), '08b') for char in secret_message) + '1111111111111110'
    msg_index = 0
    stego_image = np.zeros_like(image, dtype=np.float32)

    for i in range(0, h, 8):
        for j in range(0, w, 8):
            block = image[i:i+8, j:j+8]
            if block.shape != (8, 8):
                continue
            dct_block = apply_dct(block.astype(np.float32))

            if msg_index < len(binary_msg):
                coeff_x, coeff_y = 4, 4
                val = int(dct_block[coeff_x, coeff_y])
                val &= ~1  # Clear LSB
                val |= int(binary_msg[msg_index])  # Set LSB
                dct_block[coeff_x, coeff_y] = val
                msg_index += 1

            stego_image[i:i+8, j:j+8] = apply_idct(dct_block)

    stego_image = np.clip(stego_image, 0, 255).astype(np.uint8)
    cv2.imwrite("stego_image.png", stego_image)
    print("Message embedded and stego image saved!")



In [2]:
def extract_message(image_path):
    stego_image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    h, w = stego_image.shape
    binary_msg = ""

    for i in range(0, h, 8):
        for j in range(0, w, 8):
            block = stego_image[i:i+8, j:j+8]
            if block.shape != (8, 8):
                continue
            dct_block = apply_dct(block.astype(np.float32))

            coeff_x, coeff_y = 4, 4
            bit = int(dct_block[coeff_x, coeff_y]) & 1
            binary_msg += str(bit)

            if binary_msg.endswith('1111111111111110'):
                # Message end detected
                binary_msg = binary_msg[:-16]
                secret_message = ''.join(chr(int(binary_msg[i:i+8], 2)) for i in range(0, len(binary_msg), 8))
                print("Extracted Message:", secret_message)
                return

    print("No valid message found.")


In [ ]:
import cv2
import numpy as np
import scipy.fftpack

def apply_dct(block):
    return scipy.fftpack.dct(scipy.fftpack.dct(block.T, norm='ortho').T, norm='ortho')

def apply_idct(block):
    return scipy.fftpack.idct(scipy.fftpack.idct(block.T, norm='ortho').T, norm='ortho')

def embed_message(image_path, message, output_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    h, w = img.shape
    img = img.astype(np.float32)

    # Convert message to binary + end delimiter
    binary_msg = ''.join(format(ord(c), '08b') for c in message)
    binary_msg += '1111111111111110'
    msg_index = 0

    for row in range(0, h - 8 + 1, 8):
        for col in range(0, w - 8 + 1, 8):
            if msg_index >= len(binary_msg):
                break

            block = img[row:row+8, col:col+8]
            dct_block = apply_dct(block)

            # Mid-frequency coefficient
            x, y = 4, 3
            bit = int(binary_msg[msg_index])

            # Set coeff to strong +ve or -ve value to represent bit
            if bit == 1:
                dct_block[x, y] = 50
            else:
                dct_block[x, y] = -50

            idct_block = apply_idct(dct_block)
            img[row:row+8, col:col+8] = idct_block
            msg_index += 1

    img = np.clip(img, 0, 255).astype(np.uint8)
    cv2.imwrite(output_path, img)
    print(f"✅ Message embedded and saved to '{output_path}'")


In [ ]:
def extract_message(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    h, w = img.shape
    img = img.astype(np.float32)

    binary_msg = ''

    for row in range(0, h - 8 + 1, 8):
        for col in range(0, w - 8 + 1, 8):
            block = img[row:row+8, col:col+8]
            dct_block = apply_dct(block)

            x, y = 4, 3
            coeff = dct_block[x, y]

            # Use thresholding to decide bit
            bit = 1 if coeff > 0 else 0
            binary_msg += str(bit)

            if binary_msg.endswith('1111111111111110'):
                binary_msg = binary_msg[:-16]
                chars = [binary_msg[i:i+8] for i in range(0, len(binary_msg), 8)]
                message = ''.join(chr(int(c, 2)) for c in chars)
                return message

    return "No hidden message found."


In [25]:
input_image = "treeBW1.png"  # Should be grayscale and large enough
output_image = "secret_image.png"
secret_text = "Hello from DCT!"

# EMBED
embed_message(input_image, secret_text, output_image)

# EXTRACT
msg = extract_message(output_image)
print("🔓 Extracted Message:", msg)


✅ Message embedded and saved to 'secret_image.png'
🔓 Extracted Message: No hidden message found.


In [3]:
import cv2
import numpy as np
import math

# Utility: Convert text to binary
def text_to_bin(text):
    return ''.join(format(ord(c), '08b') for c in text)

# Utility: Convert binary to text
def bin_to_text(binary):
    chars = [binary[i:i+8] for i in range(0, len(binary), 8)]
    return ''.join(chr(int(b, 2)) for b in chars)

# Embed message in image using DCT
def embed_message(image_path, message, output_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    h, w = img.shape
    img = np.float32(img)
    
    bin_msg = text_to_bin(message)
    bin_msg += '1111111111111110'  # EOF marker (16 bits)
    
    idx = 0
    for row in range(0, h, 8):
        for col in range(0, w, 8):
            if idx >= len(bin_msg):
                break
            block = img[row:row+8, col:col+8]
            if block.shape[0] < 8 or block.shape[1] < 8:
                continue
            dct_block = cv2.dct(block)
            coeff = dct_block[3, 3]
            coeff_int = int(round(coeff))
            coeff_int = (coeff_int & ~1) | int(bin_msg[idx])
            dct_block[3, 3] = float(coeff_int)
            idct_block = cv2.idct(dct_block)
            img[row:row+8, col:col+8] = np.clip(idct_block, 0, 255)
            idx += 1
        if idx >= len(bin_msg):
            break

    result = np.uint8(img)
    cv2.imwrite(output_path, result)
    print(f"✅ Message embedded and saved to {output_path}")


# Extract message from stego image
def extract_message(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    h, w = img.shape
    img = np.float32(img)

    binary = ''
    for row in range(0, h, 8):
        for col in range(0, w, 8):
            block = img[row:row+8, col:col+8]
            if block.shape[0] < 8 or block.shape[1] < 8:
                continue
            dct_block = cv2.dct(block)
            coeff = dct_block[3, 3]
            bit = int(round(coeff)) & 1
            binary += str(bit)
            if binary[-16:] == '1111111111111110':
                message_bin = binary[:-16]
                try:
                    return bin_to_text(message_bin)
                except:
                    return "[ERROR] Failed to decode message."
    return "[ERROR] No EOF marker found."


In [14]:
if __name__ == "__main__":
    # Embed a short message
    # embed_message("treeBW1.png", "Hidden in DCT!", "stego.png")

    # Extract it
    print("Extracted message:", extract_message("stego.png"))


TypeError: 'int' object is not subscriptable

In [23]:
import cv2
import numpy as np
import math
import os

# --- Constants ---
DCT_COEFF_ROW = 4  # Changed from 3
DCT_COEFF_COL = 4  # Changed from 3
# --- End Constants ---

# Utility: Convert text to binary
def text_to_bin(text):
    """Converts a string to its 8-bit binary representation."""
    return ''.join(format(ord(c), '08b') for c in text)

# Utility: Convert binary to text
def bin_to_text(binary):
    """Converts a binary string (multiple of 8 bits) back to text."""
    if len(binary) % 8 != 0:
        print(f"[Warning] Binary string length ({len(binary)}) is not a multiple of 8. Possible data corruption.")
        binary = binary[:-(len(binary) % 8)]
        if not binary: return "[ERROR] Invalid binary data after trimming."

    chars = [binary[i:i+8] for i in range(0, len(binary), 8)]
    try:
        return ''.join(chr(int(b, 2)) for b in chars)
    except ValueError as e:
        return f"[ERROR] Failed to decode character from binary: {e}"
    except Exception as e:
        return f"[ERROR] Unknown error during binary to text conversion: {e}"


# Embed message in image using DCT with Bias Adjustment
def embed_message(image_path, message, output_path):
    """Embeds a message using DCT LSB modification at specified coefficient."""
    if not os.path.exists(image_path):
        print(f"[ERROR] Input image not found at: {image_path}")
        return False

    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        print(f"[ERROR] Failed to read image: {image_path}")
        return False

    h, w = img.shape
    if h % 8 != 0 or w % 8 != 0:
        print(f"[Warning] Image dimensions ({h}x{w}) not multiple of 8. Trimming to ({h - h % 8}x{w - w % 8}).")
        h = h - h % 8
        w = w - w % 8
        img = img[:h, :w]

    img_float = np.float32(img)

    bin_msg = text_to_bin(message)
    bin_msg += '1111111111111110'  # EOF marker

    max_bits = (h // 8) * (w // 8)
    if len(bin_msg) > max_bits:
        print(f"[ERROR] Message + EOF ({len(bin_msg)} bits) too long for image capacity ({max_bits} bits).")
        return False

    idx = 0
    stego_img = img_float.copy()
    clip_count = 0
    mismatch_count = 0
    BIAS = 0.15 # Keep bias for now

    print("Starting embedding process (using DCT coeff [{}, {}])...".format(DCT_COEFF_ROW, DCT_COEFF_COL))

    for row in range(0, h, 8):
        for col in range(0, w, 8):
            if idx >= len(bin_msg): break

            block_for_processing = stego_img[row:row+8, col:col+8].copy()
            bit_to_embed = int(bin_msg[idx])

            # --- Embed Step with Bias ---
            dct_block = cv2.dct(block_for_processing)
            # === Use the specified coefficient ===
            coeff = dct_block[DCT_COEFF_ROW, DCT_COEFF_COL]
            # ====================================
            coeff_int = int(round(coeff))

            new_coeff_int = (coeff_int & ~1) | bit_to_embed
            target_float = float(new_coeff_int)

            if bit_to_embed == 1:
                final_coeff_val = target_float + BIAS
            else:
                final_coeff_val = target_float - BIAS

            # === Set the specified coefficient ===
            dct_block[DCT_COEFF_ROW, DCT_COEFF_COL] = final_coeff_val
            # =====================================
            idct_block = cv2.idct(dct_block)
            # --- End Embed Step ---

            # --- Check Clipping ---
            min_val, max_val = np.min(idct_block), np.max(idct_block)
            needs_clipping = min_val < 0 or max_val > 255
            if needs_clipping:
                clip_count += 1
            clipped_block = np.clip(idct_block, 0, 255)
            # --- End Check Clipping ---

            # --- Immediate Check Read ---
            read_block_float = np.float32(clipped_block)
            read_dct = cv2.dct(read_block_float)
            # === Read the specified coefficient ===
            read_coeff = read_dct[DCT_COEFF_ROW, DCT_COEFF_COL]
            # ======================================
            read_coeff_int = int(round(read_coeff))
            read_bit = read_coeff_int & 1
            match = (read_bit == bit_to_embed)
            if not match:
                mismatch_count += 1
                print(f"  MISMATCH: B({row},{col}) I={idx} Bit={bit_to_embed} "
                      f"O={coeff:.1f} R={coeff_int} N(target)={new_coeff_int} "
                      f"FinalCoeff={final_coeff_val:.2f} C={'Y' if needs_clipping else 'N'} E={read_bit} M=FAIL")
            # --- End Check Read ---

            stego_img[row:row+8, col:col+8] = clipped_block
            idx += 1
        if idx >= len(bin_msg): break

    print(f"\nEmbedding process complete. Total bits processed: {idx}")
    print(f"Blocks requiring clipping: {clip_count} ({clip_count*100.0/idx:.1f}%)")
    print(f"Immediate check mismatches: {mismatch_count} ({mismatch_count*100.0/idx:.1f}%)")

    if mismatch_count > 0:
        print("[WARNING] Some bits did not survive the embed->check cycle.")

    if idx < len(bin_msg):
         print(f"[ERROR] Could not embed the full message.")
         return False

    result = np.uint8(stego_img)
    print("Attempting to save the image...")
    success = cv2.imwrite(output_path, result)
    if success:
        print(f"✅ Image processed and saved to {output_path}")
        return True
    else:
        print(f"[ERROR] Failed to write output image to {output_path}")
        return False


# Extract message from stego image
def extract_message(image_path):
    """Extracts message using DCT LSB retrieval at specified coefficient."""
    if not os.path.exists(image_path):
        print(f"[ERROR] Stego image not found at: {image_path}")
        return "[ERROR] Stego image not found."

    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        print(f"[ERROR] Failed to read stego image: {image_path}")
        return "[ERROR] Failed to read stego image."

    h, w = img.shape
    if h % 8 != 0 or w % 8 != 0:
        print(f"[Warning] Extract dimensions ({h}x{w}) not multiple of 8. Trimming.")
        h = h - h % 8
        w = w - w % 8
        img = img[:h, :w]

    img_float = np.float32(img)

    binary = ''
    eof_marker = '1111111111111110'
    eof_len = len(eof_marker)
    total_blocks = (h // 8) * (w // 8)
    blocks_processed = 0

    print("Starting extraction process (using DCT coeff [{}, {}])...".format(DCT_COEFF_ROW, DCT_COEFF_COL))

    for row in range(0, h, 8):
        for col in range(0, w, 8):
            block = img_float[row:row+8, col:col+8]
            blocks_processed += 1

            dct_block = cv2.dct(block)
            # === Use the specified coefficient ===
            coeff = dct_block[DCT_COEFF_ROW, DCT_COEFF_COL]
            # =====================================
            bit = int(round(coeff)) & 1
            binary += str(bit)

            if len(binary) >= eof_len and binary[-eof_len:] == eof_marker:
                message_bin = binary[:-eof_len]
                print(f"\nℹ️ EOF marker found after extracting {len(binary)} bits.")
                return bin_to_text(message_bin)

    print(f"\n[ERROR] No EOF marker found after processing all {blocks_processed} blocks ({len(binary)} bits).")
    return "[ERROR] No EOF marker found."


# Main execution block
if __name__ == "__main__":
    # --- Configuration ---
    INPUT_IMAGE = "treeBW1.png" # Replace with your input image
    OUTPUT_IMAGE = "stego_coeff44.png"   # Changed output name
    MESSAGE = "This is a secret message hidden using DCT!" # Keep consistent
    # --- End Configuration ---

    if not os.path.exists(INPUT_IMAGE):
        print(f"Input image '{INPUT_IMAGE}' not found.")
        dummy_h, dummy_w = 256, 256
        print(f"Creating a dummy {dummy_h}x{dummy_w} grayscale image: {INPUT_IMAGE}")
        dummy_img = np.random.randint(0, 256, (dummy_h, dummy_w), dtype=np.uint8)
        cv2.imwrite(INPUT_IMAGE, dummy_img)

    print("-" * 30)
    print("--- Running Embedding ---")
    print("-" * 30)
    embedding_successful = embed_message(INPUT_IMAGE, MESSAGE, OUTPUT_IMAGE)

    if embedding_successful:
        print("-" * 30)
        print("\n--- Running Extraction ---")
        print("-" * 30)
        extracted_msg = extract_message(OUTPUT_IMAGE)
        print("-" * 30)
        print(f"\nFinal Extracted Message: >>> {extracted_msg} <<<")
        print("-" * 30)
    else:
        print("-" * 30)
        print("\n--- Extraction Skipped ---")
        print("-" * 30)
        print("Extraction skipped because embedding failed or reported errors.")

------------------------------
--- Running Embedding ---
------------------------------
Starting embedding process (using DCT coeff [4, 4])...

Embedding process complete. Total bits processed: 352
Blocks requiring clipping: 71 (20.2%)
Immediate check mismatches: 0 (0.0%)
Attempting to save the image...
✅ Image processed and saved to stego_coeff44.png
------------------------------

--- Running Extraction ---
------------------------------
Starting extraction process (using DCT coeff [4, 4])...

[ERROR] No EOF marker found after processing all 4096 blocks (4096 bits).
------------------------------

Final Extracted Message: >>> [ERROR] No EOF marker found. <<<
------------------------------


In [21]:
import cv2
import numpy as np

def encode_dct(img, message, channel=0, min_gap=30):
    img = img.astype(np.float32)
    height, width = img.shape[:2]
    block_size = 8

    # Encode message with 16-bit header
    message_bytes = message.encode('utf-8')
    message_len = len(message_bytes)
    if message_len > 65535:
        raise ValueError("Message too long!")

    message_bits = f'{message_len:016b}' + ''.join(f'{byte:08b}' for byte in message_bytes)

    max_capacity = ((width // block_size - 1) * (height // block_size - 1))
    if len(message_bits) > max_capacity:
        raise ValueError("Message too large for image!")

    planes = cv2.split(img)
    i = 0

    for x in range(1, width // block_size):
        for y in range(1, height // block_size):
            if i >= len(message_bits):
                break

            px = (x - 1) * block_size
            py = (y - 1) * block_size

            block = planes[channel][py:py+block_size, px:px+block_size]
            dct = cv2.dct(block)

            a = dct[6, 7]
            b = dct[5, 1]
            bit = int(message_bits[i])

            if bit == 1:
                if a <= b + min_gap:
                    a = b + min_gap
            else:
                if b <= a + min_gap:
                    b = a + min_gap

            dct[6, 7] = a
            dct[5, 1] = b

            idct = cv2.idct(dct)
            planes[channel][py:py+block_size, px:px+block_size] = np.clip(idct, 0, 255)

            i += 1

    stego = cv2.merge(planes)
    return stego.astype(np.uint8)

def decode_dct(img, channel=0):
    img = img.astype(np.float32)
    height, width = img.shape[:2]
    block_size = 8

    planes = cv2.split(img)
    bits = []

    for x in range(1, width // block_size):
        for y in range(1, height // block_size):
            px = (x - 1) * block_size
            py = (y - 1) * block_size

            block = planes[channel][py:py+block_size, px:px+block_size]
            dct = cv2.dct(block)

            a = dct[6, 7]
            b = dct[5, 1]

            bits.append('1' if a > b else '0')

    # Read length (first 16 bits)
    length_bits = ''.join(bits[:16])
    msg_len = int(length_bits, 2)

    # Now decode exact number of bits = msg_len * 8
    total_bits_needed = 16 + msg_len * 8
    if total_bits_needed > len(bits):
        raise ValueError("Image does not contain enough data.")

    msg_bits = bits[16:16 + msg_len * 8]

    chars = []
    for i in range(0, len(msg_bits), 8):
        byte = ''.join(msg_bits[i:i+8])
        chars.append(chr(int(byte, 2)))

    return ''.join(chars)

# Example usage
if __name__ == "__main__":
    # ENCODING
    image = cv2.imread("treeBW.png")
    secret_message = "This is cipher text!"
    encoded_img = encode_dct(image, secret_message)
    cv2.imwrite("stego.png", encoded_img)

    # DECODING
    stego = cv2.imread("stego.png")
    hidden = decode_dct(stego)
    print("Hidden Message:", hidden)


Hidden Message: This is chher text!
